In [1]:
import os
import folium
import json
import pandas as pd
import numpy as np

# Estimativa de tempo de espera por corrida uber, nos bairros da cidade de natal
---

In [2]:
geo_json_natal = json.load(open('geojson/natal.geojson',encoding='UTF-8'))

In [3]:

estimativasData = pd.read_csv('data/estimates.csv', 
                              parse_dates=True, index_col=0,
                              names=['Hora_coleta', 
                                     'Bairro', 
                                    'Tipo Uber',
                                    'Tempo_espera'])
estimativasData['Tempo_espera'] = estimativasData['Tempo_espera']/60

mediaEstimativaData = (estimativasData.pivot_table(index='Bairro',values='Tempo_espera')).reset_index()

In [4]:
mUberTime = folium.Map(
    location=[-5.801000, -35.2094400],
    zoom_start=12,
    tiles='Stamen Terrain'
)


threshold_scale = np.linspace(mediaEstimativaData['Tempo_espera'].min(),
                              mediaEstimativaData['Tempo_espera'].max(), 6, dtype=int).tolist()


mUberTime.choropleth(
    geo_data=geo_json_natal,
    data=mediaEstimativaData,
    columns=['Bairro', 'Tempo_espera'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    legend_name='Tempo de Espera Uber',
    highlight=True,
    threshold_scale = threshold_scale,
    )

mUberTime

# Estimativa de preços do uber nos bairros da cidade de natal
---

In [5]:
#leitura do geojson com os 10 bairros sorteados
geo_10_neibhorhood = json.load(open('geojson/bairros_sorteados.geojson',encoding='UTF-8'))

In [6]:
pricesData = pd.read_csv('data/prices.csv', names=
                         ['Tempo_coleta', 'Origem', 'Destino', 'Tipo_uber', 'Distancia', 
                          'High_price', 'Low_price', 'Duration', 'Estimate', 'Currency'])
pricesData['Mean_price'] = (pricesData['High_price'] + pricesData['Low_price'])/2.0
pricesMeanData = (pricesData.pivot_table(index='Destino', values='Mean_price', fill_value=0)).reset_index(level=0)

In [7]:
mUberPrices = folium.Map(
    location=[-5.801000, -35.2094400],
    zoom_start=12,
    tiles='Stamen Terrain'
)


threshold_scale = np.linspace(pricesMeanData['Mean_price'].min(),
                              pricesMeanData['Mean_price'].max(), 6, dtype=int).tolist()



mUberPrices.choropleth(
    geo_data=geo_10_neibhorhood,
    data=pricesMeanData,
    columns=['Destino', 'Mean_price'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    legend_name='Media de preços por bairro',
    highlight=True,
    threshold_scale = threshold_scale,
    )

mUberPrices